# SciTeX DSP - Digital Signal Processing

This notebook demonstrates the signal processing capabilities of SciTeX's DSP module.

The `scitex.dsp` module provides tools for:
- Signal filtering and preprocessing
- Spectral analysis and transforms
- Time-frequency analysis
- Signal generation and manipulation

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scitex as stx
from scipy import signal

# Setup
np.random.seed(42)
print(f"SciTeX version: {stx.__version__}")

## 1. Signal Generation

SciTeX provides utilities for generating common test signals.

In [ ]:
# Generate demo signals
sig_sin, time, fs = stx.dsp.demo_sig(sig_type="sin", freq=10, duration=2, fs=1000)
sig_chirp, _, _ = stx.dsp.demo_sig(sig_type="chirp", freq=[5, 50], duration=2, fs=1000)
sig_noise, _, _ = stx.dsp.demo_sig(sig_type="noise", duration=2, fs=1000)

print(f"Signal shape: {sig_sin.shape}")
print(f"Sampling frequency: {fs} Hz")
print(f"Duration: {len(time)/fs} seconds")

In [ ]:
# Visualize generated signals
fig, axes = stx.plt.subplots(3, 1, figsize=(12, 8))

# Plot first 1000 samples for clarity
t_plot = time[:1000]

axes[0].plot(t_plot, sig_sin[0, 0, :1000])
axes[0].set_xyt("", "Amplitude", "Sine Wave (10 Hz)")
axes[0].grid(True, alpha=0.3)

axes[1].plot(t_plot, sig_chirp[0, 0, :1000])
axes[1].set_xyt("", "Amplitude", "Chirp Signal (5-50 Hz)")
axes[1].grid(True, alpha=0.3)

axes[2].plot(t_plot, sig_noise[0, 0, :1000], alpha=0.7)
axes[2].set_xyt("Time (s)", "Amplitude", "White Noise")
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
stx.io.save(fig, "./figures/demo_signals.png", symlink_from_cwd=True)
plt.show()

## 2. Filtering Operations

SciTeX provides various filtering options for signal preprocessing.

In [ ]:
# Create a noisy signal
clean_signal = np.sin(2*np.pi*10*time) + 0.5*np.sin(2*np.pi*50*time)
noisy_signal = clean_signal + 0.5*np.random.randn(len(time))

# Apply different filters
lowpass_filtered = stx.dsp.filt.lowpass(noisy_signal, fs=fs, cutoff=30)
highpass_filtered = stx.dsp.filt.highpass(noisy_signal, fs=fs, cutoff=5)
bandpass_filtered = stx.dsp.filt.bandpass(noisy_signal, fs=fs, bands=[[8, 12]])  # Around 10 Hz

print("Filtering complete!")
print(f"Original signal energy: {np.var(noisy_signal):.3f}")
print(f"Lowpass filtered energy: {np.var(lowpass_filtered):.3f}")
print(f"Bandpass filtered energy: {np.var(bandpass_filtered[0]):.3f}")

In [ ]:
# Visualize filtering results
fig, axes = stx.plt.subplots(4, 1, figsize=(12, 10))

# Show first 500 samples
t_plot = time[:500]

axes[0].plot(t_plot, noisy_signal[:500], alpha=0.7)
axes[0].set_xyt("", "Amplitude", "Original Noisy Signal")
axes[0].grid(True, alpha=0.3)

axes[1].plot(t_plot, lowpass_filtered[:500], color='orange')
axes[1].set_xyt("", "Amplitude", "Lowpass Filtered (< 30 Hz)")
axes[1].grid(True, alpha=0.3)

axes[2].plot(t_plot, highpass_filtered[:500], color='green')
axes[2].set_xyt("", "Amplitude", "Highpass Filtered (> 5 Hz)")
axes[2].grid(True, alpha=0.3)

axes[3].plot(t_plot, bandpass_filtered[0][:500], color='red')
axes[3].set_xyt("Time (s)", "Amplitude", "Bandpass Filtered (8-12 Hz)")
axes[3].grid(True, alpha=0.3)

plt.tight_layout()
stx.io.save(fig, "./figures/filtering_comparison.png", symlink_from_cwd=True)
plt.show()

## 3. Spectral Analysis

Analyze frequency content of signals using various methods.

In [ ]:
# Power Spectral Density
freqs, psd = stx.dsp.psd(noisy_signal, fs=fs)

# Create multi-component signal for spectral analysis
test_signal = (
    np.sin(2*np.pi*5*time) +           # 5 Hz component
    0.7*np.sin(2*np.pi*15*time) +     # 15 Hz component  
    0.5*np.sin(2*np.pi*30*time) +     # 30 Hz component
    0.2*np.random.randn(len(time))     # Noise
)

freqs_test, psd_test = stx.dsp.psd(test_signal, fs=fs)

print(f"Frequency resolution: {freqs[1] - freqs[0]:.2f} Hz")
print(f"Max frequency: {freqs[-1]:.1f} Hz")

In [ ]:
# Plot spectral analysis
fig, axes = stx.plt.subplots(2, 2, figsize=(12, 8))

# Time domain signals
axes[0,0].plot(time[:1000], noisy_signal[:1000], alpha=0.7)
axes[0,0].set_xyt("Time (s)", "Amplitude", "Noisy Signal")
axes[0,0].grid(True, alpha=0.3)

axes[0,1].plot(time[:1000], test_signal[:1000], color='orange')
axes[0,1].set_xyt("Time (s)", "Amplitude", "Multi-component Signal")
axes[0,1].grid(True, alpha=0.3)

# Frequency domain (PSD)
axes[1,0].semilogy(freqs, psd)
axes[1,0].set_xyt("Frequency (Hz)", "Power", "PSD - Noisy Signal")
axes[1,0].set_xlim(0, 100)
axes[1,0].grid(True, alpha=0.3)

axes[1,1].semilogy(freqs_test, psd_test, color='orange')
axes[1,1].axvline(x=5, color='red', linestyle='--', alpha=0.7, label='Expected peaks')
axes[1,1].axvline(x=15, color='red', linestyle='--', alpha=0.7)
axes[1,1].axvline(x=30, color='red', linestyle='--', alpha=0.7)
axes[1,1].set_xyt("Frequency (Hz)", "Power", "PSD - Multi-component Signal")
axes[1,1].set_xlim(0, 100)
axes[1,1].legend()
axes[1,1].grid(True, alpha=0.3)

plt.tight_layout()
stx.io.save(fig, "./figures/spectral_analysis.png", symlink_from_cwd=True)
plt.show()

## 4. Time-Frequency Analysis

Analyze signals that change frequency content over time.

In [ ]:
# Create a chirp signal (frequency sweep)
t_chirp = np.linspace(0, 2, 2000)
f_start, f_end = 10, 100
chirp_signal = signal.chirp(t_chirp, f_start, t_chirp[-1], f_end, method='linear')

# Add some amplitude modulation
am_signal = chirp_signal * (1 + 0.5 * np.sin(2*np.pi*2*t_chirp))

# Compute spectrogram
f_spec, t_spec, Sxx = signal.spectrogram(am_signal, fs=1000, nperseg=256)

print(f"Chirp signal: {f_start} Hz to {f_end} Hz over {t_chirp[-1]} seconds")
print(f"Spectrogram shape: {Sxx.shape}")
print(f"Time resolution: {t_spec[1] - t_spec[0]:.3f} s")
print(f"Frequency resolution: {f_spec[1] - f_spec[0]:.2f} Hz")

In [ ]:
# Plot time-frequency analysis
fig, axes = stx.plt.subplots(2, 1, figsize=(12, 8))

# Time domain
axes[0].plot(t_chirp, am_signal)
axes[0].set_xyt("Time (s)", "Amplitude", "Amplitude Modulated Chirp Signal")
axes[0].grid(True, alpha=0.3)

# Time-frequency representation
im = axes[1].pcolormesh(t_spec, f_spec, 10*np.log10(Sxx), shading='gouraud')
axes[1].set_xyt("Time (s)", "Frequency (Hz)", "Spectrogram (dB)")
axes[1].set_ylim(0, 150)

# Add colorbar
cbar = plt.colorbar(im, ax=axes[1])
cbar.set_label('Power (dB)')

plt.tight_layout()
stx.io.save(fig, "./figures/time_frequency_analysis.png", symlink_from_cwd=True)
plt.show()

## 5. Hilbert Transform and Analytic Signal

Extract instantaneous amplitude and phase information.

In [ ]:
# Create AM-FM signal
t_short = np.linspace(0, 1, 1000)
carrier_freq = 20
mod_freq = 3

# Amplitude modulation
amplitude = 1 + 0.5 * np.sin(2*np.pi*mod_freq*t_short)

# Frequency modulation  
frequency = carrier_freq + 5*np.sin(2*np.pi*2*t_short)
phase = 2*np.pi*np.cumsum(frequency)*t_short[1]

# Combined AM-FM signal
am_fm_signal = amplitude * np.sin(phase)

# Apply Hilbert transform
analytic_signal = stx.dsp.hilbert(am_fm_signal)
inst_amplitude = np.abs(analytic_signal)
inst_phase = np.angle(analytic_signal)
inst_frequency = np.diff(np.unwrap(inst_phase)) / (2*np.pi*t_short[1])

print(f"Original amplitude range: {amplitude.min():.2f} to {amplitude.max():.2f}")
print(f"Estimated amplitude range: {inst_amplitude.min():.2f} to {inst_amplitude.max():.2f}")
print(f"Original frequency range: {frequency.min():.1f} to {frequency.max():.1f} Hz")
print(f"Estimated frequency range: {inst_frequency.min():.1f} to {inst_frequency.max():.1f} Hz")

In [ ]:
# Plot Hilbert analysis results
fig, axes = stx.plt.subplots(3, 1, figsize=(12, 10))

# Original signal with extracted amplitude
axes[0].plot(t_short, am_fm_signal, 'b-', alpha=0.7, label='AM-FM Signal')
axes[0].plot(t_short, inst_amplitude, 'r-', linewidth=2, label='Instantaneous Amplitude')
axes[0].plot(t_short, -inst_amplitude, 'r-', linewidth=2)
axes[0].set_xyt("", "Amplitude", "Signal and Envelope")
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Instantaneous phase
axes[1].plot(t_short, np.unwrap(inst_phase))
axes[1].set_xyt("", "Phase (rad)", "Instantaneous Phase")
axes[1].grid(True, alpha=0.3)

# Instantaneous frequency
axes[2].plot(t_short[:-1], inst_frequency, 'g-', linewidth=2, label='Estimated')
axes[2].plot(t_short, frequency, 'k--', label='True Frequency')
axes[2].set_xyt("Time (s)", "Frequency (Hz)", "Instantaneous Frequency")
axes[2].legend()
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
stx.io.save(fig, "./figures/hilbert_analysis.png", symlink_from_cwd=True)
plt.show()

## 6. Signal Normalization and Preprocessing

Common preprocessing steps for signal analysis.

In [ ]:
# Create test signals with different properties
sig1 = np.sin(2*np.pi*10*time) + 5  # DC offset
sig2 = 10 * np.sin(2*np.pi*10*time)  # Large amplitude
sig3 = np.sin(2*np.pi*10*time) + 0.1*np.random.randn(len(time))  # With noise

# Apply different normalization methods
sig1_norm = stx.dsp.norm(sig1, method='zscore')
sig2_norm = stx.dsp.norm(sig2, method='minmax')
sig3_norm = stx.dsp.norm(sig3, method='robust')

print("Original signals:")
print(f"Sig1 - Mean: {np.mean(sig1):.2f}, Std: {np.std(sig1):.2f}")
print(f"Sig2 - Mean: {np.mean(sig2):.2f}, Std: {np.std(sig2):.2f}")
print(f"Sig3 - Mean: {np.mean(sig3):.2f}, Std: {np.std(sig3):.2f}")

print("\nNormalized signals:")
print(f"Sig1 (z-score) - Mean: {np.mean(sig1_norm):.2f}, Std: {np.std(sig1_norm):.2f}")
print(f"Sig2 (minmax) - Min: {np.min(sig2_norm):.2f}, Max: {np.max(sig2_norm):.2f}")
print(f"Sig3 (robust) - Mean: {np.mean(sig3_norm):.2f}, Std: {np.std(sig3_norm):.2f}")

In [ ]:
# Visualize normalization effects
fig, axes = stx.plt.subplots(3, 2, figsize=(12, 10))

t_plot = time[:200]

# Before normalization
axes[0,0].plot(t_plot, sig1[:200])
axes[0,0].set_xyt("", "Amplitude", "Original (DC Offset)")
axes[0,0].grid(True, alpha=0.3)

axes[1,0].plot(t_plot, sig2[:200])
axes[1,0].set_xyt("", "Amplitude", "Original (Large Amplitude)")
axes[1,0].grid(True, alpha=0.3)

axes[2,0].plot(t_plot, sig3[:200])
axes[2,0].set_xyt("Time (s)", "Amplitude", "Original (With Noise)")
axes[2,0].grid(True, alpha=0.3)

# After normalization
axes[0,1].plot(t_plot, sig1_norm[:200], color='orange')
axes[0,1].set_xyt("", "Amplitude", "Z-score Normalized")
axes[0,1].grid(True, alpha=0.3)

axes[1,1].plot(t_plot, sig2_norm[:200], color='orange')
axes[1,1].set_xyt("", "Amplitude", "Min-Max Normalized")
axes[1,1].grid(True, alpha=0.3)

axes[2,1].plot(t_plot, sig3_norm[:200], color='orange')
axes[2,1].set_xyt("Time (s)", "Amplitude", "Robust Normalized")
axes[2,1].grid(True, alpha=0.3)

plt.tight_layout()
stx.io.save(fig, "./figures/signal_normalization.png", symlink_from_cwd=True)
plt.show()

## 7. Resampling and Decimation

Change sampling rates while preserving signal content.

In [ ]:
# Create high-frequency sampled signal
fs_high = 2000
t_high = np.arange(0, 1, 1/fs_high)
signal_high = np.sin(2*np.pi*50*t_high) + 0.3*np.sin(2*np.pi*150*t_high)

# Resample to different rates
fs_low = 500
signal_resampled = stx.dsp.resample(signal_high, fs_high, fs_low)
t_low = np.arange(len(signal_resampled)) / fs_low

print(f"Original: {len(signal_high)} samples at {fs_high} Hz")
print(f"Resampled: {len(signal_resampled)} samples at {fs_low} Hz")
print(f"Compression ratio: {len(signal_high)/len(signal_resampled):.1f}x")

In [ ]:
# Compare original and resampled signals
fig, axes = stx.plt.subplots(3, 1, figsize=(12, 8))

# Time domain comparison
sample_range = slice(0, 200)

axes[0].plot(t_high[sample_range], signal_high[sample_range], 'b-', label=f'Original ({fs_high} Hz)')
axes[0].plot(t_low[:50], signal_resampled[:50], 'ro-', markersize=4, label=f'Resampled ({fs_low} Hz)')
axes[0].set_xyt("", "Amplitude", "Time Domain Comparison")
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Frequency domain - original
freqs_orig, psd_orig = stx.dsp.psd(signal_high, fs=fs_high)
axes[1].semilogy(freqs_orig, psd_orig)
axes[1].axvline(x=fs_low/2, color='red', linestyle='--', label='Nyquist frequency')
axes[1].set_xyt("", "Power", f"Original Signal PSD ({fs_high} Hz)")
axes[1].set_xlim(0, 300)
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# Frequency domain - resampled
freqs_resamp, psd_resamp = stx.dsp.psd(signal_resampled, fs=fs_low)
axes[2].semilogy(freqs_resamp, psd_resamp, color='orange')
axes[2].set_xyt("Frequency (Hz)", "Power", f"Resampled Signal PSD ({fs_low} Hz)")
axes[2].set_xlim(0, 300)
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
stx.io.save(fig, "./figures/signal_resampling.png", symlink_from_cwd=True)
plt.show()

## 8. Practical Example: EEG-like Signal Processing

A complete workflow for processing simulated neural signals.

In [ ]:
# Simulate multi-channel EEG-like data
fs_eeg = 250  # Typical EEG sampling rate
duration = 10  # 10 seconds
n_channels = 4
t_eeg = np.arange(0, duration, 1/fs_eeg)

# Simulate different brain rhythms
delta = np.sin(2*np.pi*2*t_eeg)      # Delta: 1-4 Hz
theta = np.sin(2*np.pi*6*t_eeg)      # Theta: 4-8 Hz  
alpha = np.sin(2*np.pi*10*t_eeg)     # Alpha: 8-13 Hz
beta = np.sin(2*np.pi*20*t_eeg)      # Beta: 13-30 Hz

# Create multi-channel data with different rhythm mixtures
eeg_data = np.zeros((n_channels, len(t_eeg)))
eeg_data[0] = 0.8*alpha + 0.3*theta + 0.2*np.random.randn(len(t_eeg))  # Dominant alpha
eeg_data[1] = 0.6*beta + 0.4*alpha + 0.2*np.random.randn(len(t_eeg))   # Beta + alpha
eeg_data[2] = 0.7*theta + 0.3*delta + 0.3*np.random.randn(len(t_eeg))  # Theta + delta
eeg_data[3] = 0.5*delta + 0.3*alpha + 0.4*np.random.randn(len(t_eeg))  # Mixed

print(f"EEG simulation: {n_channels} channels, {fs_eeg} Hz, {duration}s")
print(f"Data shape: {eeg_data.shape}")

In [ ]:
# Process each frequency band
bands = {
    'Delta': [1, 4],
    'Theta': [4, 8],
    'Alpha': [8, 13],
    'Beta': [13, 30]
}

# Extract power in each band for each channel
band_power = {}
for band_name, (low, high) in bands.items():
    filtered = stx.dsp.filt.bandpass(eeg_data, fs=fs_eeg, bands=[[low, high]])
    power = np.mean(filtered[0]**2, axis=1)  # Average power per channel
    band_power[band_name] = power

# Create power summary
power_df = stx.pd.force_df(band_power)
power_df.index = [f'Channel {i+1}' for i in range(n_channels)]

print("\nPower in each frequency band:")
print(power_df.round(3))

In [ ]:
# Visualize EEG analysis
fig = plt.figure(figsize=(15, 10))
gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)

# Raw EEG traces
ax1 = fig.add_subplot(gs[0, :])
for ch in range(n_channels):
    ax1.plot(t_eeg[:1000], eeg_data[ch, :1000] + ch*3, label=f'Ch {ch+1}')
ax1.set_xlabel('Time (s)')
ax1.set_ylabel('Amplitude')
ax1.set_title('Simulated Multi-Channel EEG')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Power spectral density for each channel
for ch in range(n_channels):
    ax = fig.add_subplot(gs[1, ch] if ch < 3 else gs[2, ch-3])
    freqs, psd = stx.dsp.psd(eeg_data[ch], fs=fs_eeg)
    ax.semilogy(freqs, psd)
    
    # Mark frequency bands
    colors = ['blue', 'green', 'orange', 'red']
    for i, (band_name, (low, high)) in enumerate(bands.items()):
        ax.axvspan(low, high, alpha=0.2, color=colors[i], label=band_name)
    
    ax.set_xlabel('Frequency (Hz)')
    ax.set_ylabel('Power')
    ax.set_title(f'Channel {ch+1} PSD')
    ax.set_xlim(0, 50)
    if ch == 0:
        ax.legend()
    ax.grid(True, alpha=0.3)

# Power comparison bar plot
ax_bar = fig.add_subplot(gs[2, 2])
x = np.arange(len(bands))
width = 0.2

for i, ch in enumerate([f'Channel {j+1}' for j in range(n_channels)]):
    values = [power_df.loc[ch, band] for band in bands.keys()]
    ax_bar.bar(x + i*width, values, width, label=ch)

ax_bar.set_xlabel('Frequency Band')
ax_bar.set_ylabel('Average Power')
ax_bar.set_title('Band Power Comparison')
ax_bar.set_xticks(x + width*1.5)
ax_bar.set_xticklabels(bands.keys())
ax_bar.legend()
ax_bar.grid(True, alpha=0.3)

stx.io.save(fig, "./figures/eeg_analysis_complete.png", symlink_from_cwd=True)
plt.show()

## Summary

This notebook demonstrated the key capabilities of SciTeX's DSP module:

1. **Signal Generation**: Creating test signals (sine, chirp, noise)
2. **Filtering**: Lowpass, highpass, and bandpass filtering
3. **Spectral Analysis**: Power spectral density computation
4. **Time-Frequency Analysis**: Spectrograms for non-stationary signals
5. **Hilbert Transform**: Extracting instantaneous amplitude and frequency
6. **Normalization**: Various signal preprocessing methods
7. **Resampling**: Changing sampling rates while preserving content
8. **Complete Workflow**: EEG-like signal processing pipeline

### Key SciTeX DSP Features:
- Unified interface for common DSP operations
- Automatic handling of multi-channel data
- Integration with SciTeX plotting and I/O systems
- Optimized for scientific data analysis workflows

### Next Steps:
- Explore `scitex.nn` for neural network-based signal processing
- Use `scitex.ai` for machine learning on signal features
- Check `scitex.stats` for statistical analysis of signal properties

In [ ]:
# Save processed data and results
results = {
    'band_power': power_df.to_dict(),
    'processing_params': {
        'fs': fs_eeg,
        'duration': duration,
        'n_channels': n_channels,
        'frequency_bands': bands
    }
}

stx.io.save(results, "./data/eeg_analysis_results.json", symlink_from_cwd=True)
stx.io.save(eeg_data, "./data/simulated_eeg_data.npy", symlink_from_cwd=True)

print("\n✅ DSP analysis complete!")
print("📊 Results saved to ./data/")
print("📈 Figures saved to ./figures/")